In [69]:
import time
import pandas as pd
import numpy as np

from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.cross_validation import train_test_split

data = pd.read_csv('version4.csv',encoding="utf-8")

#preu_park_noinclos
#preu_park_noinclos=((data['hasParkingSpace']!=data['isParkingSpaceIncludedInPrice']).astype(float))
#data=pd.concat((data,preu_park_noinclos),axis=1)
#data.drop("isParkingSpaceIncludedInPrice",axis=1)

data.columns

Index([u'rooms', u'bathrooms', u'exterior', u'hasLift', u'size', u'floor',
       u'numPhotos', u'hasVideo', u'longitude', u'latitude', u'price',
       u'showAddress', u'hasParkingSpace', u'isParkingSpaceIncludedInPrice',
       u'priceperarea', u'type_chalet', u'type_chalet_independantHouse',
       u'type_chalet_semidetachedHouse', u'type_chalet_terracedHouse',
       u'type_flat', u'type_flat_duplex', u'type_flat_penthouse',
       u'type_flat_studio', u'good', u'newdevelopment', u'renew'],
      dtype='object')

In [70]:
data.shape

(5385, 26)

In [71]:
#Applying price outlier function

def price_outlier(data, thresold):
    y=data['price']
    m = np.median(y)
    abs_dev = np.abs(y - m)
    y_mad=np.median(abs_dev)
    control_var=abs_dev/y_mad
    data["control_var"]=control_var
    data = data[data['control_var']<thresold]
    data = data.drop(['control_var'], axis=1)
    return data
    
data=price_outlier(data, 2.5)

In [72]:
data.shape

(4601, 26)

In [73]:
y =data['price']
X = data.drop(['price'], axis=1)
#X=np.array(X)
#y=np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [74]:
#Random Forest Regressor
#min_samples_leaf estimate

start_time = time.time()
# Set the parameters by cross-validation
#tuned_parameters = [{'bootstrap': [True, False],'criterion': ["gini", "entropy"],'n_estimators': [10, 50, 100, 200, 500]}]
tuned_parameters = [{'bootstrap': [True],
                     'n_estimators': [900]}]
#scores = ['precision', 'recall']

#scores = ['precision']
#scores = ['f1']

#print "Tuning hyper-parameters for %s" % score
print
clf = GridSearchCV(RandomForestRegressor(random_state=44, 
                                         n_jobs=-1,
                                         max_depth=10,
                                         max_features='auto'), 
                   tuned_parameters, 
                   cv=10, 
                   #scoring='neg_mean_absolute_error'
                   scoring = 'r2'
                   )
clf.fit(X_train, y_train)
print "Best parameters set found on training set:"
print
print clf.best_params_ 
print
y_true, y_pred = y_test, clf.predict(X_test)
print "Mean Absolute Error:"
print
print metrics.mean_absolute_error(y_true, y_pred)
print
print "Grid scores"
print
print clf.grid_scores_
print
print "The model is trained on the training set. The scores are computed on the testing set."
print("--- %s seconds ---" % (time.time() - start_time))


Best parameters set found on training set:

{'n_estimators': 900, 'bootstrap': True}

Mean Absolute Error:

231.626045645

Grid scores

[mean: 0.59147, std: 0.03603, params: {'n_estimators': 900, 'bootstrap': True}]

The model is trained on the training set. The scores are computed on the testing set.
--- 130.253000021 seconds ---
